In [32]:
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.feature_selection import mutual_info_classif

In [2]:
iris = datasets.load_iris()

In [3]:
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']

In [4]:
#Function to find label for a value
#if MIN_Value <=val < (m + Mean_Value) / 2 then it is assigned label a
#if (m + Mean_Value) <=val < Mean_Value then it is assigned label b
#if (Mean_Value) <=val < (Mean_Value + MAX_Value)/2 then it is assigned label c
#if (Mean_Value + MAX_Value)/2 <=val <= MAX_Value  then it is assigned label d

def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a, b, c, d
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

In [5]:
#Convert all columns to labelled data
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')
df

,sl,sw,pl,pw,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,5.1,3.5,1.4,0.2,b,c,a,a
1,4.9,3.0,1.4,0.2,a,b,a,a
2,4.7,3.2,1.3,0.2,a,c,a,a
3,4.6,3.1,1.5,0.2,a,c,a,a
4,5.0,3.6,1.4,0.2,a,c,a,a
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,c,b,c,d
146,6.3,2.5,5.0,1.9,c,a,c,d
147,6.5,3.0,5.2,2.0,c,b,c,d
148,6.2,3.4,5.4,2.3,c,c,d,d


In [6]:
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)

In [7]:
set(df['sl_labeled'])

{'a', 'b', 'c', 'd'}

In [68]:
def build_tree(df, y, unused_features):
    if(set(y[df])==1):
        return
    # Base case: If there are no unused features, stop
    if len(unused_features) == 0:
        return 
    # Select the best feature
    best_feature = None
    min_mistakes = float('inf')
    #finding the best features to split upon
    for f in unused_features:
        possible_values = set(df[f])
        for val in possible_values:
            subset_indices = (df[f] == val)
            subset_y = y[subset_indices]
            common_y = subset_y.value_counts().idxmax()
            mistakes = sum(subset_y != common_y)

            if mistakes < min_mistakes:
                min_mistakes = mistakes
                best_feature = f
                best_val = val

    # Remove the best feature from the unused features
    unused_features.remove(best_feature)

    print("Best feature, unused_feature",best_feature,unused_features)

    # Loop over possible values of the best feature
    for val in df[best_feature].unique():
        subset_indices = (df[best_feature] == val)
        subset_df = df[subset_indices]
        subset_y = y[subset_indices]

        # Call build_tree recursively for the subset
        build_tree(subset_df, subset_y, unused_features.copy())


In [69]:
y = pd.DataFrame(iris.target)
unused_features = set(df.columns)
# print(set(y)==1)
build_tree(df, y, unused_features)

Best feature, unused_feature sw_labeled {'sl_labeled', 'pw_labeled', 'pl_labeled'}
Best feature, unused_feature sl_labeled {'pw_labeled', 'pl_labeled'}
Best feature, unused_feature pl_labeled {'pw_labeled'}
Best feature, unused_feature pw_labeled set()
Best feature, unused_feature pl_labeled {'pw_labeled'}
Best feature, unused_feature pw_labeled set()
Best feature, unused_feature pl_labeled {'pw_labeled'}
Best feature, unused_feature pw_labeled set()
Best feature, unused_feature pw_labeled set()
Best feature, unused_feature pl_labeled {'pw_labeled'}
Best feature, unused_feature pw_labeled set()
Best feature, unused_feature pw_labeled set()
Best feature, unused_feature sl_labeled {'pw_labeled', 'pl_labeled'}
Best feature, unused_feature pl_labeled {'pw_labeled'}
Best feature, unused_feature pw_labeled set()
Best feature, unused_feature pl_labeled {'pw_labeled'}
Best feature, unused_feature pw_labeled set()
Best feature, unused_feature pw_labeled set()
Best feature, unused_feature pl_lab